In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
import panel as pn

from icat.model import Model
from icat.anchors import DictionaryAnchor
import icat

icat.initialize(offline=True)

In [ ]:
from sklearn.datasets import fetch_20newsgroups

train = fetch_20newsgroups(subset="train")
test = fetch_20newsgroups(subset="test")

train_df = pd.DataFrame({"text": train["data"]})
test_df = pd.DataFrame({"text": test["data"]})

In [ ]:
train.keys()

In [ ]:
model = Model(train_df, 'text')

In [ ]:
model.view

In [ ]:
model.anchor_list.add_anchor_type(icat.DictionaryAnchor)
model.anchor_list.add_anchor_type(icat.TFIDFAnchor, color="#FF00FF")

In [ ]:
model.anchor_list._populate_anchor_types_col()
model.anchor_list._populate_example_anchor_types_dropdown()

In [ ]:
model.anchor_list.table.expanded = []


In [ ]:
model.anchor_list.possible_anchor_types

In [ ]:
from icat.table import TableContentsTemplate

In [ ]:
TableContentsTemplate()

In [ ]:
import ipyvuetify as v

pn.Row(v.TextField(label="hiello"))

In [ ]:
v.TextField(label="hiello")

In [ ]:
model.data.filtered_df.sort_index(ascending=False)

In [ ]:
model.data.table.options

In [ ]:
model.save("testingmodel")

In [ ]:
model.load("testingmodel")

In [ ]:
model.data.active_data

In [ ]:
model.anchor_list.save("notebooks/testdataaa/anchorlist")

In [ ]:
model.anchor_list.load("notebooks/testdataaa/anchorlist")

In [ ]:
model.anchor_list.table._set_processing("DictionaryAnchor00228", True)


In [ ]:
model.anchor_list.table.items

In [ ]:
model.anchor_list.anchors[0]

In [ ]:
model.anchor_list.anchors[0].widget.children

In [ ]:
model.data.update_trigger

In [ ]:
model.data.width

In [ ]:
model.data.data_tabs.v_model

In [ ]:
model.view.anchorviz.anchors

In [ ]:
model.anchor_list.anchors

In [ ]:
model.anchor_list.add_anchor(DictionaryAnchor(anchor_name="news", keywords=["news"]))
# model.view.anchorviz.modify_anchor(0, "theta", 0.0)

indices = model.data.active_data[(model.data.active_data["_news"] > 0) & (model.data.active_data.index.isin(model.data.sample_indices))].index
for index in list(indices[0:10]):
    model.data.apply_label(index, 1)
    
model.data.apply_label(model.data.sample_indices[0], 0)

In [ ]:
model.anchor_list.add_anchor(DictionaryAnchor(anchor_name="fake news", keywords=["news"], weight=2.0))

In [ ]:
model.anchor_list.add_anchor(DictionaryAnchor(anchor_name="gun", keywords=["gun"]))
indices = model.data.active_data[(model.data.active_data["_gun"] > 0) & (model.data.active_data.index.isin(model.data.sample_indices))].index
model.data.apply_label(indices[0], 0)

In [ ]:

model.view.histograms.hist_global.layout.object = model.view.histograms.hist_global.get_vega_graph()

In [ ]:
model.data.active_data.loc[model.data.sample_indices, [model.data.prediction_col, model.data.label_col]]

In [ ]:
model.view.histograms.hist_global

In [ ]:
model.view.histograms.hist_global.set_data(model.data.active_data.loc[0:10, ["_pred", "_label"]])

In [ ]:
model.data.active_data

In [ ]:
# model.data.active_data.iloc[:10].to_dict(orient='record')
import numpy as np
np.linspace(0.0, 1.0, 51)

In [ ]:
from functools import partial
# how to manually bin data
df = model.data.active_data[["_pred", "_label"]]
bins = [0, .1, .2, .3, .4, .5, .6, .7, .8, .9, 1.0]  # TODO: make 50 bins instead
labels = bins[:-1]
hist_df = pd.DataFrame({"count": pd.cut(df._pred, bins=bins, labels=labels).value_counts()})
hist_df["bin"] = hist_df.index
hist_df["bin_end"] = hist_df["bin"].astype(float) + .1
hist_df["bin_end"] = hist_df["bin_end"].apply(round, ndigits=2)
hist_df.to_dict(orient='records')
# df._pred.cut(bins)

In [ ]:
#, bin=alt.Bin(maxbins=50))  \
import altair as alt
chart = alt.Chart(hist_df) \
    .mark_bar(align='left') \
    .encode(
        x=alt.X("bin:Q", title="", bin='binned', scale=alt.Scale(domain=[0, 1]), axis=alt.Axis(labels=True, labelColor="white", domainColor="white", gridColor="#444")),
        x2="bin_end:Q",
        y=alt.Y('count:Q', title="", axis=alt.Axis(labels=False, domainColor="white", gridColor="#444")),
        color=alt.condition("datum.bin >= 0.5", if_true={"value": "#2196F3"}, if_false={"value": "#FB8C00"}),
    ) \
    .configure(background="transparent") \
    .properties(width='container', height=150)

view = pn.panel(chart)
view

In [ ]:
chart = alt.Chart(model.data.active_data[["_pred", "_label"]].iloc[:10]) \
    .transform_bin('pred_binned', field='_pred') \
    .mark_bar(align='left') \
    .encode(
        x=alt.X("pred_binned:Q", title="", scale=alt.Scale(domain=[0, 1]), axis=alt.Axis(labels=True, labelColor="white", domainColor="white", gridColor="#444")),
        x2="pred_binned_end",
        y=alt.Y('count()', title="", axis=alt.Axis(labels=False, domainColor="white", gridColor="#444")),
        color=alt.condition("datum.pred_binned >= 0.5", if_true={"value": "#2196F3"}, if_false={"value": "#FB8C00"}),
    ) \
    .configure(background="transparent") \
    .properties(width='container', height=150)

view = pn.panel(chart)
# view

In [ ]:
view

In [ ]:
view.object = alt.Chart(model.data.active_data[["_pred", "_label"]].iloc[:5000]) \
    .mark_bar() \
    .encode(
        alt.X("_pred", bin=True),
        y='count()'
    )

In [ ]:
from icat.histogram import Histogram

h = Histogram("_pred", width=400)
pn.Row(h)

In [ ]:
h.set_data(model.data.active_data[["_pred", "_label"]].iloc[:500])

In [ ]:
import altair as alt
chart = alt.Chart(hist_df) \
    .mark_bar(align='left') \
    .encode(
        x=alt.X("bin:Q", title="", bin='binned', scale=alt.Scale(domain=[0, 1]), axis=alt.Axis(labels=True, labelColor="white", domainColor="white", gridColor="#444")),
        x2="bin_end:Q",
        y=alt.Y('count:Q', title="", axis=alt.Axis(labels=False, domainColor="white", gridColor="#444")),
        color=alt.condition("datum.bin >= 0.5", if_true={"value": "#2196F3"}, if_false={"value": "#FB8C00"}),
    ) \
    .configure(background="transparent") \
    .properties(width='container', height=150)

view = pn.panel(chart)
view

In [ ]:
h.layout.param.trigger("object")

In [ ]:
h.layout.object = h.get_vega_graph()

In [ ]:
h.data = model.data.active_data[["_pred", "_label"]].iloc[:10].to_dict(orient="record")

In [ ]:
h.update_vega()

In [ ]:
h.data

---

In [ ]:
lite = {
    "$schema": "https://vega.github.io/schema/vega-lite/v5.json",
    "usermeta": {"embedOptions": {"renderer": "svg"}},
    "background": {
        "fill":  "transparent",
    },
    "width": 400,
    "height": 100,
    "data":   {
        "name": "primary",
        # "values": model.data.active_data[["_pred", "_label"]].iloc[:10].to_dict(orient="record"),
        "values": model.data.active_data[["_pred", "_label"]].to_dict(orient="record"),
    },
    "transform": [
        {
            "bin": {"maxbins": 50, "extent": [0.0, 1.0]},
            "field": "_pred",
            "as": "pred_binned",
        },
    ],
    "mark": "bar",
    "encoding": {
        "x": {
            "field": "pred_binned",
            "bin": {
                "binned": True,
                # "step": 0.02,
                "extent": [0.0, 1.0],
            },
            "extent": [0.0, 1.0],
            "title": "",
            "scale": {
                "domain": [0.0, 1.0],
            },
        },
        "x2": {"field": "pred_binned_end"},
        "y": {
            "aggregate": "count",
            "title": "",
            "axis": {
                "labels": False,
            }
        },
        "color": {
            "field": "pred_binned",
            "type": "nominal",
            "condition": [
                {"test": "datum.pred_binned >= 0.5", "value": "#2196F3"},
                {"test": "datum.pred_binned < 0.5", "value": "#FB8C00"},
            ],
        },
    },
    "config": { "legend": { "disable": True } },
}

vgl_pane = pn.panel(lite, theme='dark')
vgl_pane

In [ ]:
vgl_pane.object = {}
type(vgl_pane)

In [ ]:
vgl_pane.debug

In [ ]:
vgl_pane.object = }

In [ ]:
lite = {
    "$schema": "https://vega.github.io/schema/vega-lite/v3.json",
    "description": "An interactive histogram for visualizing a univariate distribution.",
    "width": 500,
    "height": 100,
    "padding": 5,
    "data": [
        {
            "name": "primary",
            "values": model.data.active_data[["_pred", "_label"]].iloc[:10].to_dict(orient='record'),
        },
        {
            "name": "binned",
            "source": "primary",
            "transform": [
                {
                    "type": "bin", "field": "_pred",
                    "extent": [0, 1],
                    "anchor": {"signal": "binOffset"},
                    "step": {"signal": "binStep"},
                    "nice": False
                },
                {
                    "type": "aggregate",
                    "key": "bin0", "groupby": ["bin0", "bin1"],
                    "fields": ["bin0"], "ops": ["count"], "as": ["count"]
                }
            ]
        }
    ],
    "scales": [
        {
          "name": "xscale",
          "type": "linear",
          "range": "width",
          "domain": [-1, 1]
        },
        {
          "name": "yscale",
          "type": "linear",
          "range": "height", "round": True,
          "domain": {"data": "binned", "field": "count"},
          "zero": True, "nice": True
        }
    ],
    "axes": [
        {"orient": "bottom", "scale": "xscale", "zindex": 1},
        {"orient": "left", "scale": "yscale", "tickCount": 5, "zindex": 1}
    ],

    "marks": [
        {
            "type": "rect",
            "from": {"data": "binned"},
            "encode": {
                "update": {
                    "x": {"scale": "xscale", "field": "bin0"},
                    "x2": {"scale": "xscale", "field": "bin1",
                           "offset": {"signal": "binStep > 0.02 ? -0.5 : 0"}},
                    "y": {"scale": "yscale", "field": "count"},
                    "y2": {"scale": "yscale", "value": 0},
                    "fill": {"value": "steelblue"}
                },
                "hover": { "fill": {"value": "firebrick"} }
            }
        },
    ],
}
vgl_pane = pn.panel(lite, height=240)
vgl_pane

In [ ]:
vegalite = {
  "$schema": "https://vega.github.io/schema/vega-lite/v3.json",
  "data": {"url": "https://raw.githubusercontent.com/vega/vega/master/docs/data/barley.json"},
  "mark": "bar",
  "encoding": {
    "x": {"aggregate": "sum", "field": "yield", "type": "quantitative"},
    "y": {"field": "variety", "type": "nominal"},
    "color": {"field": "site", "type": "nominal"}
  }
}
vgl_pane = pn.panel(vegalite, height=240)
vgl_pane

---

In [ ]:
model.predict(test_df)

In [ ]:
model.feature_names()

In [ ]:
model.training_data[model.feature_names()]

In [ ]:
indices = model.data.active_data[(model.data.active_data["_news"] > 0) & (model.data.active_data.index.isin(model.data.sample_indices))].index
for index in list(indices[0:10]):
    model.data.apply_label(index, 1)
    
model.data.apply_label(model.data.sample_indices[0], 0)

In [ ]:
model.view.anchorviz.selectedDataPointID

In [ ]:
model.view.anchorviz.data

In [ ]:
model.data.active_data

In [ ]:
model2 = Model(test_df, "text")
model2.view

In [ ]:
model.data.table.contents.labeled3

In [ ]:
model.view.anchorviz.anchors

In [ ]:
model.data.selected_indices

In [ ]:
[thing for thing in model.view.anchorviz.data if thing["id"] == 9582]

In [ ]:
model.data.active_data.iloc[9582].text

In [ ]:
# is this normalization being done wrong?
total = 0
for thing in model.view.anchorviz.data:
    total += thing["weights"]["DictionaryAnchor00157"]
total

In [ ]:
model.data.active_data

In [ ]:
# model.view.anchorviz.data

In [ ]:
model.data.active_data

In [ ]:
model.anchor_list.anchors_dataframe

In [ ]:
model.anchor_list.anchors

In [ ]:
model.view.anchorviz

In [ ]:
from ipyanchorviz import AnchorViz
from ipywidgets import HBox, Output

In [ ]:
av = AnchorViz()
out = Output()
def say_thing(content):
    with out:
        print(content)
av.on_anchor_add(say_thing)
HBox([av, out])

In [ ]:
av._anchor_add_callbacks("test")

In [ ]:
model.data.active_data

In [ ]:
model.data.sample_indices

In [ ]:
model.data.selected_indices